# Important note!

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your GT login and the GT logins of any of your collaborators below. (The GT logins are worth 1 point per notebook, so don't miss the opportunity to get a free point!)

In [ ]:
YOUR_ID = "" # Please enter your GT login, e.g., "rvuduc3" or "gtg911x"
COLLABORATORS = [] # list of strings of your collaborators' IDs

In [ ]:
import re

RE_CHECK_ID = re.compile (r'''[a-zA-Z]+\d+|[gG][tT][gG]\d+[a-zA-Z]''')
assert RE_CHECK_ID.match (YOUR_ID) is not None

collab_check = [RE_CHECK_ID.match (i) is not None for i in COLLABORATORS]
assert all (collab_check)

del collab_check
del RE_CHECK_ID
del re

**Jupyter / IPython version check.** The following code cell verifies that you are using the correct version of Jupyter/IPython.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

## Strategy: Convert the PDE to an ODE

Given the conceptual model of the PDE, how do we build a "simulator" for it? One idea is to try to find a way to convert the system into a system of ordinary differential equations (ODEs), for which can then simply invoke a black-box ODE solver, such as `odeint()`, just like before. Recall that such a solver can compute approximate numerical solutions to the system,

$$\begin{eqnarray}
  \dfrac{d}{dt}\vec{y}(t)
    \equiv
      \dfrac{d}{dt}
      \left(\begin{array}{c}
        y_0(t) \\
        y_1(t) \\
        \vdots \\
        y_{m-1}(t)
      \end{array}\right)
    & = &
      \left(\begin{array}{c}
        f_0(t, \vec{y}) \\
        f_1(t, \vec{y}) \\
        \vdots \\
        f_{m-1}(t, \vec{y})
      \end{array}\right)
    \equiv
      \vec{f}(t, \vec{y}).
\end{eqnarray}$$

Let's adopt this strategy by finding a way to transform the original 1-D diffusion model PDE into this form.

In [ ]:
import numpy as np
import scipy as sp
from scipy.integrate import odeint # Goal: Find a way to reuse this black-box!

**Discretizing the PDE.** One technique to convert a PDE to a system of ODEs is to apply an _approximation_ known as the _finite differencing method_, or _FDM_. FDM consists of two steps.

_Step 1_) Replace one or more derivatives by approximations expected to be valid under small perturbations. For example, you could try replacing the second-order spatial derivative in $x$ with a _centered difference approximation_,

$$\begin{eqnarray}
  \dfrac{\partial^2 p(x,t)}
        {\partial x^2}
    & \approx &
      \dfrac{p(x+\Delta x, t) - 2 p(x, t) + p(x-\Delta x, t)}
            {(\Delta x)^2}.
\end{eqnarray}$$

_Step 2_) _Discretize_ the corresponding independent variable(s). That means you pick a variable and replace it with samples at a discrete set of points.

In this case, you approximated the second $x$ derivative so you would discretize $x$. In particular, suppose the system is defined on the finite interval $x \in [0, \lambda_0]$. Let's replace this continuous domain by a grid of $m$ evenly-spaced points. That is, let $\Delta x \equiv \dfrac{\lambda_0}{m-1}$ and let the grid consist of the $m$ points, $\{x_0, x_1, \ldots, x_{m-1}\}$, where

$$\begin{eqnarray}
  x_i & \equiv & i \Delta x = i \dfrac{\lambda_0}{m-1}.
\end{eqnarray}$$

For instance, suppose $\lambda_0 = 2$ and you choose to divide this interval into $m=21$ points.

In [ ]:
def create_grid (lambda_0, m):
    return np.linspace (0, lambda_0, m)

LAMBDA_0 = 2.0
M = 21
DX = LAMBDA_0 / (M-1)

X = create_grid (LAMBDA_0, M)
print ("Finite domain, X[:], is\n", X)

Since we are replacing $x$ by the $m$ samples $\{x_0, x_1, \ldots, x_{m-1}\}$, let's introduce a set of $m$ new state variables to represent the value of the solution $p(x,t)$ at those points:

$$\begin{eqnarray}
  p_i(t) & \equiv & p(x_i, t),
\end{eqnarray}$$

where $0 \leq i \leq m$.

In doing so, you would naturally need to translate any initial conditions for the original system into initial conditions for each of these new state variables.

For instance, suppose the random walker starts in the center of the domain, which in this example so far is $x = 1$ since $\lambda_0 = 2$. Then the probability mass at time $t$ should be concentrated in a spike at the center, i.e., $p(x=1, 0)$. This corresponds to $p_{10}(0)$ in our example where $m=21$.

**Exercise 1** (2 points). Implement a function that returns a NumPy array corresponding to a probability distribution for a "centered impulse," i.e., one whose mass is concentrated at the center. The domain should have `m` points and the cell width is `dx`.

> How should you handle the cases of `m` being even versus odd?

In [ ]:
def create_centered_impulse (m, dx):
    assert (m >= 1) and (dx > 0)
    y = np.zeros (m)
    # YOUR CODE HERE
    raise NotImplementedError()
    return y

In [ ]:
# By the way, here is a handy function to display the grid of values.

import matplotlib.pyplot as plt
%matplotlib inline

def show_grid (x, p):
    plt.plot (x, p, 'o-')
    
x_test_odd = create_grid (2.5, 11)
y_test_odd = create_centered_impulse (11, 0.25)

x_test_even = create_grid (2.25, 10)
y_test_even = create_centered_impulse (10, 0.25)

plt.figure (figsize=[12, 5])

plt.subplot (1, 2, 1)
show_grid (x_test_odd, y_test_odd)
plt.axis ([-0.25, 2.75, -0.1, 4.1])
plt.title ('p_i(0): 11 points in [0, 2.5]')
plt.xlabel ('x_i')

plt.subplot (1, 2, 2)
show_grid (x_test_even, y_test_even)
plt.axis ([-0.25, 2.75, -0.1, 4.1])
plt.title ('p_i(0): 10 points in [0, 2.25]')
plt.xlabel ('x_i')

assert y_test_odd[5] == 4.0
assert np.isclose (np.sum (y_test_odd*0.25), 1.0, atol=1e-15)
assert np.all (y_test_even[4:5] == 2.0)
assert np.isclose (np.sum (y_test_even*0.25), 1.0, atol=1e-15)
print ("\n(Passed.)")

**System of ODEs.** The net result of the preceding two steps is a system of ODEs that approximates the original PDE:

$$
\begin{eqnarray}
  \dfrac{d}{dt} \left(\begin{array}{c}
                   p_0(t) \\
                   p_1(t) \\
                   \vdots \\
                   p_{m-1}(t)
                \end{array}\right)
    & = &
      \dfrac{\gamma}{(\Delta x)^2}\left(\begin{array}{c}
        p_{-1}(t) - 2p_0(t) + p_1(t) \\
        p_{0}(t) - 2p_1(t) + p_2(t) \\
        \vdots \\
        p_{m-3}(t) - 2p_{m-2}(t) + p_{m-1}(t) \\
        p_{m-2}(t) - 2p_{m-1}(t) + p_{m}(t)
      \end{array}\right).
\end{eqnarray}
$$

**Boundary conditions.** However, there's a hitch with this system: $p_{-1}(t)$ or $p_{m}(t)$ _do not exist!_

Indeed, these are outside the boundaries of our domain. Thus, we need to figure out what to replace them with. And you have many potential options.

For instance, one option is the case of "leaky boundaries," meaning stuff escapes at the boundaries. At the right-hand side, $i=m-1$, recall that the stuff that leaves is proportional to $p_m - p_{m-1}$; therefore, to have all the mass at $m-1$ exit we could set $p_m=0$. Setting $p_{-1}=0$ would accomplish a similar feat at the left boundary. The resulting equations at the boundaries would be

$$
\begin{eqnarray}
      \frac{d}{dt} p_0 & = & \frac{\gamma}{\Delta x^2} (-2 p_0 + p_1) \\
  \frac{d}{dt} p_{m-1} & = & \frac{\gamma}{\Delta x^2} (p_{m-2} - 2 p_{m-1}).
\end{eqnarray}
$$


A different option might be "infinite walls," meaning nothing can escape. In that case, we want the change at, say, the right-hand side, which is $p_m - p_{m-1}$, to be zero. Setting $p_m = p_{m-1}$ would accomplish that. Applying similar reasoning at the left would make the boundary equations,

$$
\begin{eqnarray}
      \frac{d}{dt} p_0 & = & \frac{\gamma}{\Delta x^2} (-p_0 + p_1) \\
  \frac{d}{dt} p_{m-1} & = & \frac{\gamma}{\Delta x^2} (p_{m-2} - p_{m-1}).
\end{eqnarray}
$$

**Exercise 2** (2 points). Implement a function that evaluates the right-hand side of the system of ODEs in the case of "leaky boundaries."

In [ ]:
def f_leaky (p, gamma, dx):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
x_test_leaky = create_grid (2.25, 10)
y_test_leaky = np.zeros (10)
y_test_leaky[0] = 1.0
y_test_leaky[4:6] = 1.0
y_test_leaky[-1] = 1.0
dy_test_leaky = f_leaky (y_test_leaky, .03125, .25)
show_grid (x_test_leaky, y_test_leaky)
show_grid (x_test_leaky, dy_test_leaky)
print (dy_test_leaky)
plt.xlabel ('x_i')
plt.legend (['y_i', 'dy_i/dt (approx)'], loc=0)
plt.axis ([-0.25, 2.5, -2.25, 1.25])
assert np.allclose (dy_test_leaky, [-1, 1/2, 0, 1/2, -1/2, -1/2, 1/2, 0, 1/2, -1], atol=1e-15)
print ("\n(Passed.)")

**Exercise 3** (2 points). Implement a function that evaluates the right-hand side of the system of ODEs in the case of "infinite walls."

In [ ]:
def f_walls (p, gamma, dx):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
x_test_walls = create_grid (2.25, 10)
y_test_walls = np.zeros (10)
y_test_walls[0] = 1.0
y_test_walls[4:6] = 1.0
y_test_walls[-1] = 1.0
dy_test_walls = f_walls (y_test_walls, .03125, .25)
show_grid (x_test_walls, y_test_walls)
show_grid (x_test_walls, dy_test_walls)
print (dy_test_walls)
plt.xlabel ('x_i')
plt.legend (['y_i', 'dy_i/dt (approx)'], loc=0)
plt.axis ([-0.25, 2.5, -2.25, 1.25])
assert np.allclose (dy_test_walls, [-1/2, 1/2, 0, 1/2, -1/2, -1/2, 1/2, 0, 1/2, -1/2], atol=1e-15)
print ("\n(Passed.)")

**Exercise 4** (2 point). Implement a simulator, `sim(f, p0, T, *args)`. It should take the following inputs:

- `f(p, *args)`: A function that evaluates the right-hand side of the ODE approximation to the PDE system.
- `p0[:]`: A NumPy array containing the initial condition at all (discretized) locations.
- `T[:]`: The set of time points at which to compute a solution.
- `*args`: Positional arguments to pass to `f()`.

The simulator should return a two-dimensional NumPy array, `P[:, :]` where `P[i, k]` is the solution at position `i` at time `T[k]`. In other words, `P.shape == (len(p0), len(T))`.

In [ ]:
def sim (f, p0, T, args):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
TMAX = 1.0
NT = 5
T = np.linspace (0, TMAX, NT)

y_sim_leaky = sim (f_leaky, y_test_leaky, T, (0.03125, 0.25))
assert y_sim_leaky.shape == (len (y_test_leaky), len (T))

plt.figure (figsize=(12,5))
for i_t in range (NT):
    show_grid (x_test_leaky, y_sim_leaky[:, i_t])
plt.legend (['t={}'.format (t) for t in T], loc=0)
    
print ("Total mass over time:\n", np.sum (y_sim_leaky*0.25, axis=0))
assert np.allclose (np.sum (y_sim_leaky, axis=0)/4, [1, 0.94456479, 0.90072522, 0.86550857, 0.83677142], atol=1e-7)
print ("\n(Passed.)")

**Conservation of mass.** If you inspect the test run above, you'll see that (probability) mass is *not* conserved! That's to be expected with leaky boundaries. The following code runs the same test but with wall boundaries. Verify that mass *is* conserved in this case and observe the difference in the solutions.

In [ ]:
y_sim_walls = sim (f_walls, y_test_walls, T, (0.03125, 0.25))
assert y_sim_walls.shape == (len (y_test_walls), len (T))

plt.figure (figsize=(12,5))
for i_t in range (NT):
    show_grid (x_test_walls, y_sim_walls[:, i_t])
plt.legend (['t={}'.format (t) for t in T], loc=0)
    
print ("Total mass over time:\n", np.sum (y_sim_walls*0.25, axis=0))
assert np.allclose (np.sum (y_sim_walls, axis=0)/4, [1, 1, 1, 1, 1], atol=1e-8)
print ("\n(Passed.)")

## Comparing the continuous and discrete random walks

Consider the fully discrete, isotropic, 1-D random walk. Recall that if the walker starts at the origin an takes $k$ steps of size $\Delta x$ each, then its position, $X_k$, is a random variable whose mean and variance are, respectively,

$$
\begin{eqnarray}
    \mathrm{E}[X_k] & = & 0 \\
  \mathrm{Var}[X_k] & = & \mathrm{E}[X_k^2 - {\underbrace{\mathrm{E}[X_k]}_{=0}}^2]
                          = \mathrm{E}[X_k^2]
                          = (\Delta x)^2 k.
\end{eqnarray}
$$

As a reminder, the actual distribution is approximately the normal shown in the following plot. The domain is taken to be $[-\frac{\lambda_0}{2}, \frac{\lambda_0}{2}]$, the step size is $\Delta x = \frac{\lambda_0}{m}$, and the number of steps is approximately $\sqrt{m}$.

In [ ]:
M = 101
LAMBDA_0 = 32
DX = LAMBDA_0 / (M-1)
NUM_STEPS = 50

VAR_X_DISCRETE = DX*DX*NUM_STEPS

X = np.linspace (-LAMBDA_0/2, LAMBDA_0/2, M)

from scipy.stats import norm
P_clt = norm.pdf (X, scale=DX*np.sqrt (NUM_STEPS))
plt.plot (X, P_clt)
plt.title ("DX = {}, NUM_STEPS = {} => Var[X_{}] = {}".format (DX,
                                                               NUM_STEPS,
                                                               NUM_STEPS,
                                                               VAR_X_DISCRETE))

In the case of continuous space and time, the distribution, $p(x, t)$, governs a continuous random variable, $\hat{X}(t)$, which also has some mean and variance.

**Exercise 5** (2 points). Suppose someone runs your simulator as follows:

```python
  p_t = sim (f_leaky, p0, [0, t], (gamma, dx))[:, 1]
```

In other words, this person provided some initial distribution, `p0`, supplied a diffusion coefficient, `gamma`, step size `dx`, requested the solution at time `t` assuming leaky boundaries, and stored just the approximate distribution at that ending time in a NumPy array, `p_t`. Further suppose that the domain of interest was `[-lambda_0/2, lambda_0/2]`.

Implement a function that estimates the variance of this distribution.

> Hint: Suppose a function $f(x)$ is sampled at $m$ equally-spaced points in the domain $[-\frac{L}{2}, \frac{L}{2}]$. Denote these points by $x_i = i \cdot \Delta x$ where $\Delta x \equiv \frac{L}{m-1}$. Then,
>
> $$
  \begin{eqnarray}
    \int_{-\frac{L}{2}}^{\frac{L}{2}} f(x) \, dx
    & \approx & \sum_{i=0}^{m} f(x_i) \cdot \Delta x.
  \end{eqnarray}
  $$

In [ ]:
def estimate_var_dist (p_t, lambda_0, dx):
    m = int (1 + lambda_0/dx)
    assert len (p_t) == m
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
var_test = estimate_var_dist (y_sim_leaky[:, -1], 2.25, 0.25)
print (var_test)
assert np.isclose (var_test, 0.388679586729, atol=1e-3)
print ("\n(Passed.)")

**Exercise 6** (3 points). Consider a discrete random walk that begins at the origin and then takes `k` steps of size `dx` each. Recall that its variance is `(dx**2) * k`.

Now suppose you wish to run your PDE simulator as

```python
p_t = sim (f_leaky, p0, [0, t], (gamma, dx))[:, 1]
```

where `gamma == 0.5*(dx**2)` and `p0` is a centered impulse on some domain `[-0.5*lambda_0, 0.5*lambda_0]`. Write some code to determine the value of `t` such that the estimated variance of `p_t` is close to `(dx**2) * k`. Store this value in a variable named, `t_match`.

The scaffolding code below will print `t_match` and superimpose the distribution implied by your solution against a normal distribution whose variance is `(dx**2)*k`.

> Note: The scaffolding code also defines the value of `dx` as `DX`, `k` as `NUM_STEPS`, `gamma` as `GAMMA`, and `(dx**2)*k` as `VAR_X_DISCRETE`.

In [ ]:
M = 101
LAMBDA_0 = 32
DX = LAMBDA_0 / (M-1)
NUM_STEPS = 25
VAR_X_DISCRETE = DX * DX * NUM_STEPS
GAMMA = 0.5 * DX * DX

# YOUR CODE HERE
raise NotImplementedError()

print ("The value of `t_match` you found is:", t_match)

In [ ]:
x = np.linspace (-LAMBDA_0/2, LAMBDA_0/2, M)
p_clt = norm.pdf (X, scale=DX*np.sqrt (NUM_STEPS))
p_0 = create_centered_impulse (M, DX)
p_pde = sim (f_leaky, p_0, [0, t_match], (GAMMA, DX))[:, 1]
plt.plot (x, p_clt, '-')
plt.plot (x, p_pde, 'o')
plt.legend (['Normal', 'PDE'], loc=0)
plt.title ('t_match = {}'.format (t_match))

v_diff = np.max (np.abs (p_clt - p_pde))
print ("Maximum absolute difference:", v_diff)
assert v_diff <= 0.01
print ("\n(Passed.)")